In [ ]:
item_creat_chart = pd.read_sql(' SELECT itemid, label, description FROM d_chartitems WHERE label like "%creati%"', conn)
item_creat_chart

In [ ]:
item_creat_lab = pd.read_sql(' SELECT itemid, test_name FROM d_labitems WHERE test_name like "%creati%"', conn)
item_creat_lab

In [ ]:
bun_table = bun_ckd.drop('hadm_id', axis=1).\
            drop('icustay_id', axis=1).drop('value', axis=1).drop('flag', axis=1).drop('valueuom', axis=1)

In [ ]:
<h2> Selecting the creatinine values for the patients with CKD </h2>
<p>In the MIMIC II dictionary the ITEMID's associated with the creatinine are as follows:
<ul>
    <li>17 - APS Serum Creatinine</li>
    <li>791 - Creatinine (0-1.3) Chemistry</li>
    <li>1129 - ZCreatinine (0-1.3) Chemistry</li>
    <li>1525 - Creatinine Chemistry</li>
    <li>2975 - URINE CREATININE</li>
    <li>3750 - Creatinine (0-0.7)	Chemistry</li>
    <li>5525 - Lytes BUN creatinine	</li>
    <li>5811 - urine creatinine</li>    
</ul>
</p>

In [2]:
### Making a dictionary for the CKD related ICD9 codes and for itemid for the various creatinine

In [ ]:
dic_ckd_icd = {"585.9":"unspecified", \
               "585.1": "Stage I", \
               "585.2": "Stage II", \
               "585.3": "Stage III", \
               "585.4": "Stage IV", \
               "585.5": "Stage V", \
               "585.6": "End-Stage Renal Disease"}

dic_itemid_desc = {"17":"Serum Creatinine", \
                   "791":"Creatinine (0-1.3)", \
                   "1129":"ZCreatinine (0-1.3)", \
                   "1525":"Creatinine Chemistry", \
                   "2975":"URINE CREATININE", \
                   "3750":"Creatinine (0-0.7)", \
                   "5525":"Lytes BUN creatinine", \
                   "5811":" urine creatinine"}

In [ ]:
set_obesity = \
pd.read_sql('SELECT DISTINCT(subject_id) \
            FROM comorbidity_scores WHERE subject_id \
            IN (SELECT DISTINCT(subject_id) FROM icd9 WHERE code IN \
                        ("585.9", "585.1", "585.2", "585.3", "585.4", "585.5", "585.6" ) AND obesity = 1.0)', conn)
set_obesity = set_obesity['subject_id'].tolist()


In [ ]:
set_drug_abuse = \
pd.read_sql('SELECT DISTINCT(subject_id) \
            FROM comorbidity_scores WHERE subject_id \
            IN (SELECT DISTINCT(subject_id) FROM icd9 WHERE code IN \
                        ("585.9", "585.1", "585.2", "585.3", "585.4", "585.5", "585.6" ) AND drug_abuse = 1.0)', conn)
set_drug_abuse = set_drug_abuse['subject_id'].tolist()


In [ ]:
set_alcohol_abuse = \
pd.read_sql('SELECT DISTINCT(subject_id) \
            FROM comorbidity_scores WHERE subject_id \
            IN (SELECT DISTINCT(subject_id) FROM icd9 WHERE code IN \
                        ("585.9", "585.1", "585.2", "585.3", "585.4", "585.5", "585.6" ) AND alcohol_abuse = 1.0)', conn)
set_alcohol_abuse = set_alcohol_abuse['subject_id'].tolist()


In [ ]:
set_liver_disease = \
pd.read_sql('SELECT DISTINCT(subject_id) \
            FROM comorbidity_scores WHERE subject_id \
            IN (SELECT DISTINCT(subject_id) FROM icd9 WHERE code IN \
                        ("585.9", "585.1", "585.2", "585.3", "585.4", "585.5", "585.6" ) AND liver_disease = 1.0)', conn)
set_liver_disease = set_liver_disease['subject_id'].tolist()

In [ ]:
set_obesity = set(set_obesity)
set_drug_abuse = set(set_drug_abuse)
set_alcohol_abuse = set(set_alcohol_abuse)
set_liver_disease = set(set_liver_disease)


In [ ]:
calc_gfr(gfr_calc_info['VALUENUM'], gfr_calc_info['AGE'], gfr_calc_info['GENDER'], gfr_calc_info['ETHNICITY']) 

In [ ]:
def calc_ccr(gender, age, weight, scr):
    """
    This function calcualtes the creatinine clearance based on the "Cockcroft and Gault formula (1973)"
    CCr = {((140–age) x weight)/(72xSCr)}x 0.85 (if female)
    
    Arguments: 
        gender: Patient's gender "male" or "female" as string
        age: Patient's age in years as integer
        weight: Patient's weight in kg as float
        scr: Patient's serum creatinine level in mg/dL as float
        
    Returns: 
        ccr: Patient's creatinine clearance in mL/min as float rounded to two decimal numbers.                       
    """
    
    if gender == 'female':
        ccr = round(((((140-age) * weight)/(72*scr))* 0.85), 2) 
    else: 
        ccr = round((((140-age) * weight)/(72*scr)), 2)
    return ccr

In [ ]:
def widget_plot(scr, age, weight):
    plt.figure(2)
    x = np.linspace(-1, 5, num=10)
    y = round((((140-age) * weight)/(72*scr)),0)
    plt.plot(x, y)
    plt.ylim(-1, 1)
    plt.show()

interactive_plot = interactive(widget_plot, scr=(-2.0, 3.0), age=(50, 80), weight=(90.0, 300.0))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

In [ ]:
class Person:

    def __init__(self, first, last):
        self.firstname = first
        self.lastname = last

    def Name(self):
        return self.firstname + " " + self.lastname

class Employee(Person):

    def __init__(self, first, last, staffnum):
        Person.__init__(self,first, last)
        self.staffnumber = staffnum

    def GetEmployee(self):
        return self.Name() + ", " +  self.staffnumber

x = Person("Marge", "Simpson")
y = Employee("Homer", "Simpson", "1007")

print(x.Name())
print(y.GetEmployee())

In [ ]:
class CareGivers:
    

    def __init__(self, firstname, lastname):
        self.firstname = firstname
        self.lastname = lastname

    def get_details(self):
         return self.firstname +" "+ self.lastname

In [ ]:
class Doctor(CareGivers):
    
    def __init__(self, first, last, _id1):
        Doctor.__init__(self,first, last, _id1)
        self._id1 = _id1

    def GetEmployeeDetails(self):
        return self.get_details() + ", " +  self._id1

In [ ]:
care_giver = CareGivers('Shilpy', 'Sharma')

doctor = Doctor('Shilpy','Sharma', '1')

print(care_giver.get_details())


In [ ]:
print(doctor.GetEmployeeDetails())

In [ ]:
from matplotlib_venn import venn3_circles, venn2, venn3
import matplotlib.pyplot as plt

comorbid_ckd = \
pd.read_sql('SELECT DISTINCT(subject_id), congestive_heart_failure, cardiac_arrhythmias, hypertension, obesity, alcohol_abuse \
            drug_abuse \
            FROM comorbidity_scores WHERE subject_id \
            IN (SELECT DISTINCT(subject_id) FROM icd9 WHERE code IN \
                        ("585.9", "585.1", "585.2", "585.3", "585.4", "585.5", "585.6" ))', conn)

comorbid_ckd.head()

heart_failure = \
pd.read_sql('SELECT DISTINCT(subject_id) \
            FROM comorbidity_scores WHERE subject_id \
            IN (SELECT DISTINCT(subject_id) FROM icd9 WHERE code IN \
                        ("585.9", "585.1", "585.2", "585.3", "585.4", "585.5", "585.6" ) AND congestive_heart_failure = 1.0)', conn)

heart_failure = heart_failure['subject_id'].tolist()

diabetes = \
pd.read_sql('SELECT DISTINCT(subject_id) \
            FROM comorbidity_scores WHERE subject_id \
            IN (SELECT DISTINCT(subject_id) FROM icd9 WHERE code IN \
                        ("585.9", "585.1", "585.2", "585.3", "585.4", "585.5", "585.6" ) AND \
                        (diabetes_uncomplicated = 1.0 OR diabetes_complicated = 1.0))', conn)

diabetes = diabetes['subject_id'].tolist()

hypertension = \
pd.read_sql('SELECT DISTINCT(subject_id) \
            FROM comorbidity_scores WHERE subject_id \
            IN (SELECT DISTINCT(subject_id) FROM icd9 WHERE code IN \
                        ("585.9", "585.1", "585.2", "585.3", "585.4", "585.5", "585.6" ) AND hypertension = 1.0)', conn)

hypertension = hypertension['subject_id'].tolist()

set_heart_failure = set(heart_failure)
set_diabetes = set(diabetes)
set_hypertension = set(hypertension)


venn3(subsets = (set_heart_failure, set_diabetes,set_hypertension), set_labels = ('Heart Failure', 'Diabetes', 'Hypertension'))


In [ ]:
tmp = test.get_venn("Heart Failure", "Diabetes", "Hypertension")
tmp = test.get_venn("Obesity", "Drug Abuse", "Alcohol Abuse")

### Looking for all the tables that exists in the MIMIC II database

In [ ]:
tables = pd.read_sql("""SHOW TABLES""", conn)
tables["Tables_in_mimic2"]

### SHOW COLUMNS to look at the columns in a table

In [ ]:
from IPython.display import display, HTML

sql_queries = [(t,"SHOW COLUMNS FROM %s"%t) for t in tables["Tables_in_mimic2"]]
for q in sql_queries:
    display(HTML("<h2>%s</h2>"%q[0]))
    display(HTML(pd.read_sql(q[1], conn).to_html()))

In [ ]:
gfr_calc_info = pd.read_excel('gfr_calc_info.xlsx')
gfr_calc_info.dropna()

### Use regular expression to separate ethnicity if it is "BLACK/AFRICAN AMERICAN" to "AFRICAN AMERICAN"

In [ ]:
def replace_black(excel_file):
    """
    This function replaces all the ethnicity that is maked "BLACK/AFRICAN AMERICAN" to "AFRICAN AMERICAN"
    Argument:  excel_file - to extract the column
    Returns: excel_file - with ethnicity replaced.
    """
    

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

from ipywidgets import interact, interactive, fixed

In [ ]:
ckd_age_counts = ckd_age["age_years"].value_counts().to_frame(name="Age Counts")
ckd_age_counts.sort_values(by="Age Counts")
ckd_age_counts.plot(kind='bar', title='Occurence of CKD based on age')

In [ ]:
eth = pd.read_sql("SELECT DISTINCT(subject_id) FROM demographic_detail WHERE subject_id \
                            IN (SELECT DISTINCT(subject_id) FROM icd9\
                            WHERE code IN \
                            ('585.9', '585.1', '585.2', '585.3', '585.4', '585.5' \
                            '585.6'))", conn)

In [ ]:
eth = pd.read_sql('SELECT * FROM icd9 WHERE code IN \
                ("585.9", "585.1", "585.2", "585.3", "585.4", "585.5",\
                "585.6" )', conn) 

In [ ]:
eth = pd.read_sql("SELECT itemid, label FROM d_demographicitems WHERE category='ETHNICITY'", conn)
eth.head()

eth = pd.read_sql("SELECT * FROM demographic_detail", conn)
eth

In [ ]:
eth = pd.read_sql("SELECT DISTINCT(subject_id), ethnicity_descr FROM demographic_detail  \
                WHERE ethnicity_descr IN ('WHITE', 'BLACK/AFRICAN AMERICAN', 'ASIAN', 'OTHER')  \
                AND subject_id \
                            IN (SELECT DISTINCT(subject_id) FROM icd9\
                            WHERE code IN \
                            ('585.9', '585.1', '585.2', '585.3', '585.4', '585.5' \
                            '585.6'))", conn)
eth.head()

In [ ]:
from project import DiseaseFrame
test = DiseaseFrame(conn)

test.get_dfs()


In [ ]:
from ipywidgets import interact, interactive, fixed
from IPython.display import clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from ipywidgets import interact, interactive, fixed

from project import DiseaseFrame

In [ ]:
icd9_codes["code"].value_counts(normalize=False)

In [ ]:
# subject_info = pd.read_sql(' SELECT subject_id, sex, dob FROM d_patients', conn)  subject_info.head()
DiseaseFrame(conn).get_patient_info().head()

In [ ]:
#123456789#123456789#123456789#123456789#123456789#123456789#123456789#12345678